In [4]:
import pandas as pd

# 6.4GB CSV 읽는 대신, 압축된 Parquet 파일을 10초 만에 로드
df_master = pd.read_parquet("df_master_preprocessed.parquet")

# CSV를 읽었을 때와 똑같이 DataFrame으로 보여짐
print(df_master.head())

print("\n--- Reviews DataFrame Info (Data Types) ---")
df_master.info()

   beer_id         username        date  \
0   271781     bluejacket74  2017-03-17   
1   125646          _dirty_  2017-12-21   
2   125646  GratefulBeerGuy  2017-12-20   
3   125646         LukeGude  2017-12-20   
4   125646          jneff33  2017-12-20   

                                                text  look  smell  taste  \
0     750 ml bottle, 2016 vintage, bottle #304 of...  4.00   4.00   4.00   
1                                                     4.50   4.50   4.50   
2     0% 16 oz can. Funny story: As I finally wal...  4.75   4.75   4.50   
3     Classic TH NEIPA. Overflowing head and bouq...  4.25   4.50   4.25   
4                                                     4.50   4.75   5.00   

   feel  overall  score  ...           availability   abv  \
0  4.25     4.00   4.03  ...  Limited (brewed once)  10.8   
1  4.50     4.50   4.50  ...               Rotating   8.2   
2  4.50     4.50   4.58  ...               Rotating   8.2   
3  4.25     4.25   4.31  ...            

In [5]:
# --- 2. 날짜 전처리 및 필터링 (핵심) ---
print("\nFiltering data for years 2013 and 2017...")

# 2-1. 'date' 컬럼(object)을 datetime 객체로 변환합니다.
#      errors='coerce'는 변환 중 오류가 발생하는 날짜(예: 'Unknown')를 NaT(결측치)로 만듭니다.
df_master['datetime'] = pd.to_datetime(df_master['date'], errors='coerce')

# 2-2. 'datetime' 컬럼에서 'year'를 추출하여 새 컬럼을 만듭니다.
df_master['year'] = df_master['datetime'].dt.year

# 2-3. (선택) 날짜 변환 후 결측치(NaT)가 있는지 확인합니다.
original_rows = df_master.shape[0]
df_master = df_master.dropna(subset=['year']) # 날짜를 알 수 없는 리뷰는 제거
new_rows = df_master.shape[0]
print(f"Removed {original_rows - new_rows} rows with invalid dates.")

# 2-4. 'year' 컬럼이 2013 또는 2017인 행만 선택합니다.
years_to_keep = [2013, 2017]
df_filtered = df_master[df_master['year'].isin(years_to_keep)].copy()

# 2-5. (선택) 메모리 확보를 위해 더 이상 필요 없는 컬럼을 제거합니다.
df_filtered = df_filtered.drop(columns=['datetime', 'date', 'text', 'notes_beer', 'notes_brewery'])

# --- 3. 결과 확인 ---
print(f"\nFiltering complete.")
print(f"Original shape: {df_master.shape} (모든 연도)")
print(f"Filtered shape: {df_filtered.shape} (2013, 2017년만)")
print("\nFiltered data 'year' counts:")
print(df_filtered['year'].value_counts())

# --- 4. (다음 단계) ---
# 이제 'df_filtered'를 사용하여 CF, Hybrid 모델 등을 구축합니다.
# X = df_filtered.drop(columns=['score', 'is_like'])
# y = df_filtered['score']


Filtering data for years 2013 and 2017...
Removed 0 rows with invalid dates.

Filtering complete.
Original shape: (8417033, 28) (모든 연도)
Filtered shape: (1863769, 23) (2013, 2017년만)

Filtered data 'year' counts:
year
2013    1267028
2017     596741
Name: count, dtype: int64
